<p align="center">
  <img src="https://github.com/droyti46/aiijc2025/blob/main/img/stage3/banner.png?raw=true" width="800">
</p>
<p align="center">
  <img src="https://github.com/droyti46/aiijc2025/blob/main/img/stage3/architecture.png?raw=true" width="800">
</p>

<p align="center"><b>[ Схема блокнота ]</b></p>

1. **Предообработка**: перед тем, как подавать изображение модели, переводим `pdf` файл в картинки `png` и изменяем разрешение до `350 dpi` (не слишком мало, модель может разглядеть, что написано, но и не слишком много, чтобы файл не весил слишком много)

2. **Запрос к модели**: была выбрана модель `Gemini-2.5-pro`. На наших русскоязычных данных Gemini показала наиболее высокую точность по сравнению с моделями, такими как `GPT5-Image` и `Qwen3 VL 235B A22B Instruct`. При этом, в промте уточняется, как правильно печатать математику.

3. **Постобработка**: заменяем все буквы `Ё` на `Е` (так скор поднялся с 0.94 до 0.95)

## 🔧 **Установка зависимостей и базовая настройка**

In [ ]:
from IPython.display import clear_output

!pip install PyMuPDF==1.24.10 pillow==10.4.0 opencv-python-headless==4.10.0.84 \
             numpy==1.26.4 requests==2.32.3 regex==2024.7.24 tqdm==4.66.5 \
             pandas==2.2.2 langchain_openai "langchain>=0.3.8" "langchain-core>=0.3.72,<1.1.0"

clear_output()

**⚠️ Внимание**: если при запуске следующей ячейки вы увидите ошибку:

```python
ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject
```

То просто перезапустите сеанс \
**Исправление:** `Среда выполнения → Перезапустить сеанс`

In [ ]:
import os
import io
import base64
import math
import requests

import fitz
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

In [ ]:
OPENROUTER_KEY = "YOUR-API-KEY"
OPENROUTER_URL = "https://openrouter.ai/api/v1"
OPENROUTER_MODEL = "google/gemini-2.5-pro"

In [ ]:
def get_key_usage():
    r = requests.get(
        "https://openrouter.ai/api/v1/key",
        headers={"Authorization": f"Bearer {OPENROUTER_KEY}"},
        timeout=20
    )
    r.raise_for_status()
    d = r.json()["data"]
    return round(float(d["usage"]), 2)

start_usage = get_key_usage()
print(f"💸 В начале пайплайна на счету ${start_usage:.2}")

💸 В начале пайплайна на счету $4.7e+02


In [ ]:
data = pd.read_csv("test_private.csv")
data.head()

,idx,task,solution
0,3,Докажите: эмпирическая функция распределения F...,NaN
1,6,Пусть \(X\sim \mathrm{Exp}(\lambda)\). Требует...,NaN
2,11,Пусть \(P\in M_n(\mathbb{F})\) удовлетворяет \...,NaN
3,13,Вычислить интеграл: $\displaystyle \int_{1/\sq...,NaN
4,14,Вычислить несобственный интеграл: $\displaysty...,NaN
...,...,...,...
95,333,"Найти площадь фигуры, ограниченной линиями: $y...",NaN
96,335,Найдите сумму ряда: $\displaystyle \frac{1}{1 ...,NaN
97,338,Вычислить определитель: \[ \begin{vmatrix} -7 ...,NaN
98,340,"Найди сумму всех целых оснований $ b > 9, $ дл...",NaN


In [ ]:
# Распаковка архива
# создать папку назначения (если её нет)
!mkdir -p "/content/test_private"
# распаковать zip в папку
!unzip -q -o "/content/test_private.zip" -d "/content/test_private"
!echo "✅ Архив распакован в: /content/test_private"

## 🧱 **Вспомогательные функции**
Используем функции из бейзлайна: создание модели, перевод pdf файла в png картинки, запрос к llm через OpenRouter

In [ ]:
def llm_init() -> ChatOpenAI:
    llm = ChatOpenAI(
        model=OPENROUTER_MODEL,
        api_key=OPENROUTER_KEY,
        base_url=OPENROUTER_URL,
        timeout = 600
    )
    return llm

In [ ]:
def pdf_to_images(pdf_path: str, dpi: int = 200):
    doc = fitz.open(pdf_path)
    zoom = dpi / 72
    mat = fitz.Matrix(zoom, zoom)
    imgs = []
    for page in doc:
        pix = page.get_pixmap(matrix=mat, alpha=False)
        b = pix.tobytes("png")
        img = Image.open(io.BytesIO(b)).convert("RGB")
        imgs.append(img)
    doc.close()
    return imgs

In [ ]:
def image_to_base64(img: Image.Image) -> str:
    buf = io.BytesIO()
    img.save(buf, format="PNG")
    return base64.b64encode(buf.getvalue()).decode("utf-8")

In [ ]:
def ocr_page_with_llm(img: Image.Image, llm: ChatOpenAI, prompt: str) -> str:
    b64 = image_to_base64(img)
    message = HumanMessage(
        content=[
            {"type": "text", "text": prompt},
            {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{b64}"}},
        ]
    )
    res = llm.invoke([message])
    return str(res.content).strip()

In [ ]:
def build_prompt(idx: int, task_text: str) -> str:
    return (
        "Тебе дано изображение страницы решения. Распознай изображение 1:1 и верни ровно один цельный фрагмент русского текста"
        "без LaTeX, без Markdown и без любой другой разметки.\n"
        "Пиши только то, что напечатано в решении. Не добавляй пояснений, не перефразируй. Но пиши по-русски: явные опечатки в словах исправляй.\n"
        "Правила формата:\n"
        "1) Ничего лишнего нельзя добавлять или что-то лишнее убирать ни в коем случае и ни при каких обстоятельствах. Слово в слово, только с математикой есть нюансы. Не продолжай рассуждения автора, даже если видно, что они пропущены, только переписывай текст с картинки.\n"
        "2) Не заменяй слова на символы и наоборот. Например, если написано «x стремится к нулю», так и пиши; если написано «x→0», пиши «x→0» Если написано 'плюс', пиши 'плюс', если '+' - пиши '+'.\n"
        "3) Сохраняй порядок изложения и абзацы как на странице; не переставляй части и не сокращай ход рассуждений. Но если какая-то часть текста зачёркнута, не пиши её.\n"
        "4) Степени писать вот так, если один символ (например): x^2, x^a и вот так, если несколько (например): x^{3/2}, 3^{200}.\n"
        "5) Аналогично с корнями: ∛5 (скобки нет, так как символ под знаком корня один),√(-10); аналогично с индексами: x_1, a_{k1}, a_{k1}x_1.\n"
        "6) И точно также с другими функциями: ∫_{x0}^x, ∫_0^π, lim_{x→0}.\n"
        "Ещё примеры правильной записи: С_{20}^2 (а не C(20, 2)), f_n'(t), |sin y|,  (sin x)/x, ∑_{k=1}^n k^2 = n(n+1)(2n+1), (a+b)/(c+d), log_2 x.\n"
        f"Ниже будет приведён текст задания. Если видишь его на картинке - смело пропускай. Текст:\n {task_text}\n\n"
    )

In [ ]:
import os, time, random, threading
from pathlib import Path
from typing import Dict, Any, List, Optional
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
from tqdm.notebook import tqdm
from IPython.display import display
import json

# --- настройки ---
MAX_WORKERS_PER_BATCH = 50
OCR_CONCURRENCY       = 12
MAX_RETRIES           = 5
BASE_DELAY            = 1.5
DPI                   = 350
OUT_CSV               = Path("/content/submission.csv")

OCR_GATE = threading.Semaphore(OCR_CONCURRENCY)

In [ ]:
def retry_sleep(att: int):
    time.sleep(BASE_DELAY * (2 ** att) + random.uniform(0, 0.35))

def chunk_df(df: pd.DataFrame, size: int):
    for start in range(0, len(df), size):
        yield df.iloc[start:start + size]

def save_records(records: List[Dict[str, Any]], path: Path):
    pd.DataFrame(records).to_csv(path, index=False)

In [ ]:
def process_row(row: pd.Series) -> Dict[str, Any]:
    """
    PDF -> изображения -> OCR страниц с ретраями.
    Печатаем первый УСПЕШНЫЙ (первая удачная попытка по каждой странице),
    в CSV пишем финальную склейку всех страниц.
    """
    llm = llm_init()

    idx = int(row["idx"])
    task_text = str(row["task"])
    pdf_path = f"test_private/test_private/{idx}.pdf"

    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"PDF не найден: {pdf_path}")

    images = pdf_to_images(pdf_path, dpi=DPI)

    page_texts: List[str] = []
    first_pass_page_texts: List[str] = []

    for img in images:
        last_err: Optional[Exception] = None
        for attempt in range(MAX_RETRIES):
            try:
                time.sleep(random.uniform(0, 0.2))
                prompt = build_prompt(idx, task_text)
                with OCR_GATE:
                    t = ocr_page_with_llm(img, llm, prompt)
                txt = t if t is not None else ""
                page_texts.append(txt)
                first_pass_page_texts.append(txt)
                break
            except json.JSONDecodeError as e:
                last_err = e
                print(f"[RETRY] json decode error idx={idx} attempt={attempt+1}/{MAX_RETRIES}")
                retry_sleep(attempt)
                continue
            except Exception as e:
                last_err = e
                print(f"[RETRY] transient idx={idx} attempt={attempt+1}/{MAX_RETRIES}: {e}")
                retry_sleep(attempt)
                continue
        else:
            page_texts.append("")
            first_pass_page_texts.append("")
            print(f"[SKIP] page failed after {MAX_RETRIES} attempts for idx={idx}: {last_err}")

    solution_text = "\n\n".join([t for t in page_texts if t])
    first_solution_text = "\n\n".join([t for t in first_pass_page_texts if t])

    return {"idx": idx, "task": task_text, "solution": solution_text, "first_solution": first_solution_text}

In [ ]:
def run_batch(batch_df: pd.DataFrame):
    futures = {}
    with ThreadPoolExecutor(max_workers=min(MAX_WORKERS_PER_BATCH, len(batch_df))) as ex:
        for _, row in batch_df.iterrows():
            futures[ex.submit(process_row, row)] = int(row["idx"])
        for fut in as_completed(futures):
            idx = futures[fut]
            try:
                rec = fut.result()
                all_records.append({"idx": rec["idx"], "task": rec["task"], "solution": rec["solution"]})
            except Exception as e:
                print(f"fail idx={idx}: {e}")
                try:
                    task_text = str(batch_df.loc[batch_df["idx"] == idx, "task"].iloc[0])
                except Exception:
                    task_text = ""
                all_records.append({"idx": idx, "task": task_text, "solution": ""})
            finally:
                save_records(all_records, OUT_CSV)
                pbar_total.update(1)

## 🔮 **Извлечение текста (основная прогонка)**

In [ ]:
needed = {"idx","task"}
missing = needed - set(data.columns)
if missing:
    raise ValueError(f"В data не хватает колонок: {missing}")

data_run = data.copy()
data_run["idx"] = pd.to_numeric(data_run["idx"], errors="coerce").astype("Int64")
if data_run["idx"].isna().any():
    raise ValueError("Есть строки с нечисловым idx")

all_records: List[Dict[str, Any]] = []

if OUT_CSV.exists():
    prev = pd.read_csv(OUT_CSV)
    if {"idx","task","solution"} <= set(prev.columns):
        done = set(pd.to_numeric(prev["idx"], errors="coerce").dropna().astype(int).tolist())
        data_run = data_run[~data_run["idx"].astype(int).isin(done)].copy()
        all_records.extend(prev.to_dict(orient="records"))

pbar_total = tqdm(total=len(data_run), desc="Total progress")

for chunk in chunk_df(data_run, MAX_WORKERS_PER_BATCH):
    run_batch(chunk)

pbar_total.close()

submission = pd.DataFrame(all_records).sort_values("idx").reset_index(drop=True)
submission.head()


Total progress:   0%|          | 0/100 [00:00<?, ?it/s]

,idx,task,solution
0,3,Докажите: эмпирическая функция распределения F...,Смотрим на фиксированное число x. Хотим понять...
1,6,Пусть \(X\sim \mathrm{Exp}(\lambda)\). Требует...,Пусть X - экспоненциальная случайная величина ...
2,11,Пусть \(P\in M_n(\mathbb{F})\) удовлетворяет \...,"Есть матрица P, которая ведёт себя как «кнопка..."
3,13,Вычислить интеграл: $\displaystyle \int_{1/\sq...,Хотим просчитать значение определённого интегр...
4,14,Вычислить несобственный интеграл: $\displaysty...,Хотим посчитать несобственный интеграл по всей...


## 🧹 **Постобработка**
Удаляем букву ё, так как она не используется

In [ ]:
# заменить 'ё' → 'е' и 'Ё' → 'Е' в колонке 'solution'
submission['solution'] = (
    submission['solution']
      .str.replace('ё', 'е', regex=False)
      .str.replace('Ё', 'Е', regex=False)
)


In [ ]:
submission.to_csv('/content/submission.csv', index=False)

In [ ]:
end_usage = get_key_usage()
print(f"💸 В конце пайплайна на счету ${end_usage:.2}")
print(f"💸 Всего потрачено ${end_usage - start_usage:.2}")

💸 В конце пайплайна на счету $4.7e+02
💸 Всего потрачено $0.0


<p align="center">
  <img src="https://github.com/droyti46/aiijc2025/blob/main/img/snr-logo.png?raw=true" width="500">
</p>